# Before We Get Started, We Must Adress Acessing The Raw Data. 

This data is massive and cannot be uploaded to GitHub.

### To Acess the Raw Data Download the Raw Data Folder Within This Google Drive. 
"https://drive.google.com/drive/folders/1fnolrZtKUfsCs28yQUW587gDe7cv3SmO?usp=drive_link"

From there you will have to ensure you have specified the local path name to that "raw_data" folder when calling the dataimport function. 

In [3]:
import pandas as pd
import numpy as np

## EIM - WDOE Dataset Cleaning

### Load The Multiple Sites Data Sets Into Global Enviroments

In [4]:
eim_vars = [
'GRG002_0',
'GRG002_1',
'BLL009',
#'FID001',
'SJF001_0', 
'SJF001_1',
'SKG003', 
'ADM002', 
'ADM001_0', 
'ADM001_1',
'SAR003_0',
'SAR003_1', 
'PSS019', 
'ADM003_0',
'ADM003_1', 
'PAH003',
]

path = "/Users/carterwebb/Desktop/5700_DBM/DBM_Kelp/" #grab your local file path and place here

In [5]:
def dataimport(eim_vars, path):


    for var in eim_vars:
        globals()[var] = pd.read_csv(path + var + ".csv")


In [6]:
dataimport(eim_vars, path) 

In [17]:
PAH003.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 30 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Study_ID                                        8068 non-null   object 
 1   Study_Name                                      8068 non-null   object 
 2   Location_ID                                     8068 non-null   object 
 3   Study_Specific_Location_ID                      8068 non-null   object 
 4   Location_Name                                   8068 non-null   object 
 5   Instrument_ID                                   8068 non-null   object 
 6   Field_Collection_Type                           8068 non-null   object 
 7   Field_Collector                                 8068 non-null   object 
 8   Time_Zone                                       8068 non-null   object 
 9   Field_Collection_Date                    

### Start Building Snake Case Data Sets
This function builds standardized snake_case datasets by iterating over a dictionary of DataFrames and renaming columns according to the rename mapping.

In [7]:
def build_eim_dfs(eim_vars, strict=True):

    rename_map = {
        "Location_ID": "lid",
        "Location_Name": "name",
        "Instrument_ID": "int_id",
        "Field_Collection_Date": "date",
        "Field_Collection_Time": "time",
        "Field_Collection_Date_Time": "date_time",
        "Depth_Value": "depth",
        "Depth_Value_Units": "dep_units",
        "Result_Parameter_Name": "parameter",
        "Result_Value": "result",
        "Result_Value_Units": "result_units",
        "Calculated_Latitude_Decimal_Degrees_NAD83HARN": "lat",
        "Calculated_Longitude_Decimal_Degrees_NAD83HARN": "lon",
    }

    clean = {}

    for var in eim_vars:

        df = globals().get(var)
        if df is None:
            raise ValueError(f"{var} not found")

        # APPLIES BELOW #
        # Flags missing columns, all the data sets should have the same syntax but 
        # As we found out through cleaning, that wasnt true. FID001 was a different syntax.
        # This is the reason for introducing this block. 
        
        if strict:
            missing = [c for c in rename_map if c not in df.columns]
            if missing:
                raise KeyError(f"{var} missing columns: {missing}")

        keep_cols = [c for c in rename_map if c in df.columns]

        d = df[keep_cols].copy()
        d = d.rename(columns=rename_map)

        clean[var] = d

    return clean

In [8]:
clean = build_eim_dfs(eim_vars)

### Lets Concat
Strip keys, stack, then melt together, resetting index for sanity. 

In [9]:
def concat_eim_clean(clean_dict, ignore_index=True):
    return pd.concat(clean_dict.values(), axis=0, ignore_index=ignore_index, sort=False)
    
eim_all = concat_eim_clean(clean)

In [16]:
eim_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4330308 entries, 0 to 4330307
Data columns (total 13 columns):
 #   Column        Dtype  
---  ------        -----  
 0   lid           object 
 1   name          object 
 2   int_id        object 
 3   date          object 
 4   time          object 
 5   date_time     object 
 6   depth         float64
 7   dep_units     object 
 8   parameter     object 
 9   result        float64
 10  result_units  object 
 11  lat           float64
 12  lon           float64
dtypes: float64(4), object(9)
memory usage: 429.5+ MB


### Stripping Unnecessary Information for Our Scope
 - First keep only temp

 - Then only 2014 - 2024 (same as kelp data)

 - Now only within the scope of kelp growth (~20m)

You may ask, why not a function here like others, I had to keep checking step by step because of the messyness of this data at this point, didnt want to loose important data or introduce dirty data like in the Snake_Case function. 

In [10]:
temp_eim = eim_all[
    eim_all["parameter"].str.contains("temperature", case=False, na=False)
].copy()

In [11]:
temp_eim["parameter"].unique()

array(['Temperature, water'], dtype=object)

The data was unsorted so we had to sort first!

In [12]:
temp_eim["date_time"] = pd.to_datetime(temp_eim["date_time"], errors="coerce")

# Index was increadibly messy, had to look up .reset_index
temp_eim = (
    temp_eim
    .sort_values(["lid", "date_time"])
    .reset_index(drop=True)
)

/var/folders/7s/t5l3pg4j2j54njys9zrdzdxr0000gn/T/ipykernel_64431/344483683.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp_eim["date_time"] = pd.to_datetime(temp_eim["date_time"], errors="coerce")


I still dont quite understand what these warnings mean, perhaps this is why date time was still object in anyalsis? This should serve as a remind to me to come back and check this.

Youll see I set it for 2014 - 2026, however, it seems only data between 2014 and 2024 are uploaded as of now, this is ok as that is same case for our kelp data. 

In [13]:
temp_eim_2014_2025 = temp_eim[
    (temp_eim["date_time"].dt.year.between(2014, 2026)) &
    (temp_eim["depth"] <= 20)
].copy()

print(
    temp_eim_2014_2025["date_time"].dt.year.min(),
    temp_eim_2014_2025["date_time"].dt.year.max()
)

print(
    temp_eim_2014_2025["depth"].min(),
    temp_eim_2014_2025["depth"].max()
)

2014 2024
0.5 20.0


In [15]:
temp_eim_2014_2025.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32513 entries, 31013 to 427762
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   lid           32513 non-null  object        
 1   name          32513 non-null  object        
 2   int_id        32513 non-null  object        
 3   date          32513 non-null  object        
 4   time          32513 non-null  object        
 5   date_time     32513 non-null  datetime64[ns]
 6   depth         32513 non-null  float64       
 7   dep_units     32513 non-null  object        
 8   parameter     32513 non-null  object        
 9   result        32513 non-null  float64       
 10  result_units  32513 non-null  object        
 11  lat           32513 non-null  float64       
 12  lon           32513 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(8)
memory usage: 3.5+ MB


## Clean temp_eim_2014_2025 df

# Moving on to NWSC - MRC Data

## Load the Two Datasets we Need
 - Kelp Data
 - Lat Lon for Sites

In [13]:
data_vars= [
   'AllBeds_Clean',
    'latlon_sites'
]

path = "/Users/carterwebb/Desktop/5700_DBM/DBM_Kelp/" #grab your local file path and place here

In [14]:
def dataimport(data_vars, path):


    for var in data_vars:
        globals()[var] = pd.read_csv(path + var + ".csv")


In [15]:
dataimport(data_vars, path) 

### Now Collapsing The Temp Data Into A Singular Value

- Adapted from previous Kelp Work last quarter

### Reason for? 

The current temperature column has many nans, however there are other temperature related columns (such as Temp Shore Edge) that have infomation in the same row. We want fill one singular value for temperature, based on the information we have.

Throughout the survey's history, collection protocol has become more grainular.

- 2016 - 2020: 1 Temperature Taken During Every Survey; df["Temp"]
- 2020 - 2022: 2 Temperature Taken During Every Survey; df["Ave Temp Shore Edge", "Ave Temp Water Edge"]
    - Important to know, these are not averages, they are recoded as such to reduce the size of the data files. 
- 2022 - 2024: 4 Temperature Taken During Every Survey; df["Temp1 Shore Edge",	Temp1 Water Edge",	"Temp2 Shore Edge",	"Temp2 Water Edge"]

We need to simplify all of these temperatures into singular values in order to do future analysis. 

### Logic/Algorithm Goal for First Portion of Funciton

 1. If Temp exists → use Temp
    
    2 Else:
       
        a. If both Ave Temp Shore Edge and Ave Temp Water Edge exist:
       
              (Ave Temp Shore Edge + Ave Temp Water Edge) / 2
       
        b. Else:
       
              ((Temp1 Shore Edge + Temp2 Shore Edge)/2  +
               (Temp1 Water Edge + Temp2 Water Edge)/2) / 2
    

This Was A Messy Hack Job, May Try To Clean Up Before The Final

### Set Survey Date to Date Time
### Lastly, Select for Only The NWSC Max Extent Data


In [23]:
def prepare_temp_max_extent(df):

    temp_cols = [
        "Temp",
        "Ave Temp Shore Edge",
        "Ave Temp Water Edge",
        "Temp1 Shore Edge",
        "Temp2 Shore Edge",
        "Temp1 Water Edge",
        "Temp2 Water Edge",
    ]

    for c in temp_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")

  
    df["Temp_Final"] = np.where(
        ~df["Temp"].isna(),
        df["Temp"],
        np.where(
            (~df["Ave Temp Shore Edge"].isna()) &
            (~df["Ave Temp Water Edge"].isna()),
            (df["Ave Temp Shore Edge"] + df["Ave Temp Water Edge"]) / 2,
            (
                ((df["Temp1 Shore Edge"] + df["Temp2 Shore Edge"]) / 2) +
                ((df["Temp1 Water Edge"] + df["Temp2 Water Edge"]) / 2)
            ) / 2
        )
    )

    
    df["Survey Date"] = pd.to_datetime(df["Survey Date"])


    
    df_max = (
        df[df["NWSC Max Ext"] == 1]
        .sort_values(["Site Code", "Survey Year", "Acres"],
                     ascending=[True, True, False])
        .drop_duplicates(subset=["Site Code", "Survey Year"], keep="first")
        .copy()
    )



    df_max["Temp_t"] = df_max["Temp_Final"]

    return df_max

### Working Now On The Lat Lon Data

In [17]:
AllBeds_Clean["Bed Name"].unique()

array(['Aiston Preserve', 'Biz Point', 'Cherry Point-Gulf Rd',
       'Clallam Bay', 'Coffin Rocks', "Ebey's Landing",
       'Freshwater Bay 1', 'Freshwater Bay 2', 'Hat Island',
       'Hoypus Point', 'Lowell', 'Lummi SW', 'North Beach East',
       'Polnell Point', 'Possession Point', 'Shannon Point East',
       'Shannon Point West'], dtype=object)

In [18]:
latlon_sites["Location"].unique()

array(['Clallam Bay', 'FWB2/Observatory Point', 'FWB3/Extension',
       'North Beach', 'Edmonds Dive Park', 'Edmonds North', 'Meadowdale',
       'Mukilteo', 'Hat Island', 'Hat Island West', 'Possession Point',
       "Ebey's", 'Polnell', 'Lowell Point Camano State Park', 'Hoypus',
       'Coffin Rocks', 'Biz Point', 'Shannon Point West',
       'Shannon Point East', 'Lummi SW', 'Aiston', 'Cherry Point/Gulf Rd',
       'Point Whitehorn', 'FWB1', 'Biz Point South', 'Lone Tree Point'],
      dtype=object)

For now we dont really mind if NaN values pop up for unshared bed locations, so a basic .map() will work

In [19]:
rename_map = {
    'Aiston': 'Aiston Preserve', 
    'Biz Point': 'Biz Point', 
    'Cherry Point/Gulf Rd': 'Cherry Point-Gulf Rd',
    'Clallam Bay': 'Clallam Bay', 
    'Coffin Rocks': 'Coffin Rocks', 
    "Ebey's": "Ebey's Landing",
    'FWB1': 'Freshwater Bay 1', 
    'FWB2/Observatory Point' : 'Freshwater Bay 2', 
    'Hat Island': 'Hat Island',
    'Hoypus': 'Hoypus Point', 
    'Lowell Point Camano State Park': 'Lowell', 
    'Lummi SW': 'Lummi SW',
    'North Beach': 'North Beach East',
    'Polnell': 'Polnell Point', 
    'Possession Point': 'Possession Point', 
    'Shannon Point East': 'Shannon Point East',
    'Shannon Point West': 'Shannon Point West'}

latlon_sites['Location'] = latlon_sites['Location'].map(rename_map)

Drop NaNs and Left Join

In [20]:
latlon_clean = (
    latlon_sites[["Location", "lat", "lon"]]
    .dropna(subset=["Location"])
    .drop_duplicates("Location")
)

AllBeds_latlon = AllBeds_Clean.merge(
    latlon_clean,
    left_on="Bed Name",
    right_on="Location",
    how="left"
)

Apply The Previous Function

In [24]:
AllBeds_latlon = prepare_temp_max_extent(AllBeds_latlon)

## Finally Building Our Large Concatenated Fact Table With EIM - WDOE & NWSC - MRC Data

In [25]:
def build_fact_table_clean(AllBeds_latlon, temp_eim_2014_2025):

    ### Prep EIM ###
    
    eim = temp_eim_2014_2025.copy()

    # datetime
    eim["date_time"] = pd.to_datetime(eim["date_time"], errors="coerce")

    # create nwsc_max in order to partition the fact sheet easier
    eim["NWSC_Max"] = 0
    
    # Drop Columns on eim
    eim = eim[[
        "lid",
        "name", 
        "date_time",
        "parameter", 
        "result", "result_units",
        "depth", "dep_units",
        "lat", "lon", 
        "NWSC_Max"
    ]].copy()


    ### Prep AllBeds ###

    beds = AllBeds_latlon.copy()

    # datetime
    beds["Survey Date"] = pd.to_datetime(beds["Survey Date"], errors="coerce")

    # drop columns and rename to match eim
    beds_clean = pd.DataFrame({
        "lid": beds["Site Code"],  # Site Code -> lid
        "name": beds["Bed Name"],     # Bed Name -> name
        "date_time": beds["Survey Date"],  # Survey Date -> date_time
        "parameter": "temperature",     # constant
        "result": beds["Temp_Final"],   # Temp_Final -> result
        "result_units": "celsius",     # constant
        "depth": 0,   # constant
        "dep_units": "m",  # constant
        "lat": beds["lat"],
        "lon": beds["lon"],
        "NWSC_Max": beds["NWSC Max Ext"]
    })


    ### Concat ###

    fact_table_clean = pd.concat([beds_clean, eim], ignore_index=True)

    return fact_table_clean

In [27]:
fact_table_clean = build_fact_table_clean(AllBeds_latlon, temp_eim_2014_2025)
fact_table_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32650 entries, 0 to 32649
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   lid           32650 non-null  object        
 1   name          32650 non-null  object        
 2   date_time     32650 non-null  datetime64[ns]
 3   parameter     32650 non-null  object        
 4   result        32632 non-null  float64       
 5   result_units  32650 non-null  object        
 6   depth         32650 non-null  float64       
 7   dep_units     32650 non-null  object        
 8   lat           32650 non-null  float64       
 9   lon           32650 non-null  float64       
 10  NWSC_Max      32650 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(5)
memory usage: 2.7+ MB


In [28]:
nwsc = fact_table_clean[fact_table_clean["NWSC_Max"] == 1]
nwsc

,lid,name,date_time,parameter,result,result_units,depth,dep_units,lat,lon,NWSC_Max
0,AISP,Aiston Preserve,2018-07-31,temperature,16.000,celsius,0.0,m,48.680355,-122.629921,1.0
1,AISP,Aiston Preserve,2019-08-01,temperature,16.000,celsius,0.0,m,48.680355,-122.629921,1.0
2,AISP,Aiston Preserve,2020-07-20,temperature,15.150,celsius,0.0,m,48.680355,-122.629921,1.0
3,AISP,Aiston Preserve,2021-07-27,temperature,14.440,celsius,0.0,m,48.680355,-122.629921,1.0
4,AISP,Aiston Preserve,2022-07-31,temperature,20.000,celsius,0.0,m,48.680355,-122.629921,1.0
...,...,...,...,...,...,...,...,...,...,...,...
132,SHPT-W,Shannon Point West,2020-09-18,temperature,11.400,celsius,0.0,m,48.507718,-122.688303,1.0
133,SHPT-W,Shannon Point West,2021-09-19,temperature,10.695,celsius,0.0,m,48.507718,-122.688303,1.0
134,SHPT-W,Shannon Point West,2022-08-12,temperature,11.945,celsius,0.0,m,48.507718,-122.688303,1.0
135,SHPT-W,Shannon Point West,2023-09-01,temperature,13.150,celsius,0.0,m,48.507718,-122.688303,1.0


In [ ]:
fact_table_clean.to_csv("/Users/carterwebb/Desktop/5700_DBM/DBM_Kelp/fact_table_clean.csv", index=False)